# Travel

In [104]:
import datetime
print(f"Last updated: {str(datetime.datetime.now())[:10]}")

Last updated: 2025-01-06


After traveling to various cultural sites for some years, I found a rule of thumb for finding good spots to visit: check if something is a UNESCO World Heritage site. Sometimes this leads to obvious suggestions (Vatican City, Paris, Amsterdam) but more often than not visiting world heritage sites has shown me sites that are very much worth visiting but not as touristy as the usual sites (e.g., Bologna, Orange, Albi).

At first, my enthousiasm led to the dream of visiting all world heritage sites in Europe, but this dream soon showed to be too ambitious. All EU countries then? Still quite a lot. Some narrowing down brought me to the following goal: visit all world heritage sites in the continental part of the original member states of the European Union. Achievable, but also ambitious, with Germany, France and Italy as the biggest obstacles. Since then, I've been using this as a heuristic in planning vacations.

(Of course, I still travel outside of these guidelines as well. Occasionally this leads to another world heritage site, and these are also included in the below map)

Lines mark seperate vacations. Lines in grey mark planned vacations, lines in light gray are trips I'd like to make someday

In [105]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from travel_plotter import journey_generator

def customwrap(s, width=10):
    s_split = s.split(' ')
    output = []
    current = ""
    for item in s_split:
        current += item + ' '
        if len(current) > width:
            output.append(current)
            current = ""
    if current != "":
        output.append(current)
    return "<br>".join(output)

def customsplit(s, length=900):
    s_split = s.split('.')
    output = ""
    for sentence in s_split:
        if len(output) + len(sentence) < length:
            output += sentence + '.'
        else:
            break
    assert len(output) < length
    return output

# Prepare dataset
df = pd.read_csv("WHC-sites.csv")
df = df[df['Years-visited'].notna()]
df['Year-last-visited'] = df['Years-visited'].apply(lambda x: x[-4:] if not x == "-1" else "Not yet visited")
df['Rating'] = df['Rating'].apply(lambda x: x if pd.notna(x) else "Not yet visited")
df['Visited'] = (df['Year-last-visited'] != 'Not yet visited')

fig = go.Figure()
for lats, lons, color, year in journey_generator(df):
    fig2 = px.line_mapbox(lat=lats, lon=lons, color_discrete_sequence=[color])
    fig2.update_layout(mapbox_style='carto-positron', hovermode=False)
    fig = go.Figure(data=fig.data + fig2.data)

df['name_en'] = df['name_en'].apply(lambda x: customwrap(x, 40))
df['short_description_en'] = df['short_description_en'].apply(lambda x: customsplit(customwrap(x, 40)))
assert not df['short_description_en'].apply(lambda x: len(x) > 900).any()

max_bound = max(abs(min(df['latitude'].astype(float))-max(df['latitude'])), abs(min(df['longitude'])-max(df['longitude']))) * 175
zoom = 11.5 - np.log(max_bound)


fig1 = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='Visited',
                       hover_name='name_en', mapbox_style='carto-positron', zoom=zoom,
                       custom_data=['Rating', 'Year-last-visited', "short_description_en"], hover_data={
                           'latitude':False,
                           'longitude':False,
                       })
fig1.update_traces(hovertemplate='<b>%{hovertext}</b><br>Rating: %{customdata[0]} <br>Last visited in: %{customdata[1]}<br><br>%{customdata[2]}<extra></extra>')
fig1.update_layout(showlegend=False, margin={'t':0,'l':0,'b':0,'r':0})

fig = go.Figure(data=fig.data + fig1.data, layout=fig1.layout)

config = {'scrollZoom': True}
fig.show(config=config)


<p style="text-align: center;">↑ Click me!</p>

In [106]:
print("Progress:")
df1 = df[
    (df['udnp_code'].str.contains('nld')) |
    (df['udnp_code'].str.contains('fra')) |
    (df['udnp_code'].str.contains('ita')) |
    (df['udnp_code'].str.contains('bel')) |
    (df['udnp_code'].str.contains('deu')) |
    (df['udnp_code'].str.contains('lux'))
]

abbrs = ["Total", 'nld', 'fra', 'ita', 'bel', 'deu', 'lux']
countries = ["Total", "Netherlands", "France", "Italy", "Belgium", "Germany", "Luxembourg"]
max_country_len = max([len(x) for x in countries])
width = 40

for abbr, country in zip(abbrs, countries):
    got = len(df1[df1['Visited'] & (df1['udnp_code'].str.contains(abbr))])
    need = len(df1[df1['udnp_code'].str.contains(abbr)])
    if abbr == "Total":
        got = len(df1[df1['Year-last-visited'] != 'Not yet visited'])
        need = len(df1)
    displays = int((got/need) * width)
    symbol = "⋆" if (got/need) == 1 else "+"
    if abbr == "Total": need = f"{need}\n"
    print(f"{country}: {' ' * (max_country_len - len(country))}" +
          f"[{symbol * displays}{'.' * (width - displays)}] " +
          f"{got} out of {need}")

print("\nNote: due to transboundary sites the sum of all countries might not match the total")

Progress:
Total:       [++++++++++..............................] 45 out of 175

Netherlands: [++++++++++++++++++++....................] 6 out of 12
France:      [++++++++++++............................] 15 out of 47
Italy:       [++++++..................................] 9 out of 60
Belgium:     [+++++++++++++++++++++++++...............] 10 out of 16
Germany:     [+++++++++...............................] 13 out of 54
Luxembourg:  [........................................] 0 out of 1

Note: due to transboundary sites the sum of all countries might not match the total
